In [240]:
import kagglehub 
import os 

#Downloads the most recent dataset from Kaggle. 
path = kagglehub.dataset_download("dhrubangtalukdar/fortune-500-companies-stock-data")
print("Path to dataset files:", path) #Just a string of where Kaggle downloads to.

data_source = os.listdir(path+ "/fortune_500_stock_data")

Path to dataset files: /Users/chrisguarino/.cache/kagglehub/datasets/dhrubangtalukdar/fortune-500-companies-stock-data/versions/1


In [241]:
import pandas as pd
ds_dict = {}

for dataset in data_source: 
    ds_dict[dataset] = pd.DataFrame(pd.read_csv(path+"/fortune_500_stock_data/"+dataset))
ds_dict

{'NVR.csv':             Date                Open               High                 Low  \
 0            NaN                 NVR                NVR                 NVR   
 1     2015-01-02    1246.97998046875            1258.75              1239.0   
 2     2015-01-05  1240.1099853515625             1255.0   1235.989990234375   
 3     2015-01-06              1250.0  1254.989990234375    1235.43994140625   
 4     2015-01-07  1250.1099853515625   1285.97998046875  1249.9300537109375   
 ...          ...                 ...                ...                 ...   
 2761  2025-12-23    7439.97998046875   7439.97998046875    7266.77978515625   
 2762  2025-12-24      7309.240234375             7435.0              7301.0   
 2763  2025-12-26    7399.02001953125     7426.240234375       7342.41015625   
 2764  2025-12-29              7380.0             7420.0     7329.7998046875   
 2765  2025-12-30    7352.56005859375             7365.0     7283.9599609375   
 
                   Close   

# Pull/Clean/Split Data

In [242]:
ibm_data = ds_dict["IBM.csv"]
ibm_data


,Date,Open,High,Low,Close,Adjusted_Close,Volume
0,NaN,IBM,IBM,IBM,IBM,IBM,IBM
1,2015-01-02,154.216064453125,156.12811279296875,153.91969299316406,154.93307495117188,98.12301635742188,5779673
2,2015-01-05,154.17782592773438,154.17782592773438,152.18930053710938,152.49522399902344,96.57909393310547,5104898
3,2015-01-06,152.64817810058594,152.92543029785156,148.34608459472656,149.2064971923828,94.49626922607422,6429448
4,2015-01-07,150.28680419921875,150.28680419921875,147.25621032714844,148.23135375976562,93.87866973876953,4918083
...,...,...,...,...,...,...,...
2761,2025-12-23,301.3399963378906,305.1300048828125,300.6499938964844,303.7799987792969,303.7799987792969,2923000
2762,2025-12-24,303.760009765625,305.1499938964844,302.29998779296875,304.55999755859375,304.55999755859375,1210600
2763,2025-12-26,304.69000244140625,305.75,303.6700134277344,305.0899963378906,305.0899963378906,2814700
2764,2025-12-29,304.6499938964844,310.0,303.75,305.739990234375,305.739990234375,4664500


In [243]:
ibm_data=ibm_data.iloc[1:].reset_index(drop=True)

In [244]:
ibm_data["Date"] = pd.to_datetime(ibm_data["Date"]).map(pd.Timestamp.toordinal)

In [245]:
ibm_data = ibm_data.apply(pd.to_numeric, errors="coerce")

In [246]:
X=ibm_data[["Open","High","Low","Close","Adjusted_Close","Volume"]]

In [247]:
y=ibm_data["Date"]

In [248]:
X

,Open,High,Low,Close,Adjusted_Close,Volume
0,154.216064,156.128113,153.919693,154.933075,98.123016,5779673
1,154.177826,154.177826,152.189301,152.495224,96.579094,5104898
2,152.648178,152.925430,148.346085,149.206497,94.496269,6429448
3,150.286804,150.286804,147.256210,148.231354,93.878670,4918083
4,149.369019,152.045883,148.709366,151.453156,95.919090,4431693
...,...,...,...,...,...,...
2760,301.339996,305.130005,300.649994,303.779999,303.779999,2923000
2761,303.760010,305.149994,302.299988,304.559998,304.559998,1210600
2762,304.690002,305.750000,303.670013,305.089996,305.089996,2814700
2763,304.649994,310.000000,303.750000,305.739990,305.739990,4664500


In [274]:
from sklearn.model_selection import train_test_split

X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3)
X_val,X_test,y_val,y_test = train_test_split(X_temp,y_temp,test_size=0.5)


In [279]:
import torch 

X_train = torch.tensor(X_train.to_numpy(),dtype=torch.float32)
y_train = torch.tensor(y_train.to_numpy(),dtype=torch.float32).view(-1, 1)
X_val   = torch.tensor(X_val.to_numpy(),dtype=torch.float32)
y_val   = torch.tensor(y_val.to_numpy(),dtype=torch.float32).view(-1, 1)
X_test  = torch.tensor(X_test.to_numpy(),dtype=torch.float32)
y_test  = torch.tensor(y_test.to_numpy(),dtype=torch.float32).view(-1, 1)



In [280]:
#What we want is to map the columns of all of the features we are tracking to 1 date.
model = torch.nn.Linear(X_train.shape[1],1)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
criterion = torch.nn.MSELoss()

# Training Loop

In [289]:
epochs = 10000

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

In [288]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
print(test_loss.item())


111561826304.0
